# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,8.68,93,100,5.65,RU,1695351568
1,1,invercargill,-46.4000,168.3500,8.90,70,87,2.24,NZ,1695351568
2,2,tiksi,71.6872,128.8694,2.41,63,96,7.93,RU,1695351569
3,3,waitangi,-43.9535,-176.5597,14.90,87,50,4.02,NZ,1695351569
4,4,bilibino,68.0546,166.4372,4.33,72,100,0.45,RU,1695351569


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
"Lng",
"Lat",
geo=True,
tiles= "OSM",
frame_width = 600,
frame_height = 600,
scale = 1,
size = "Humidity",
color = "City"
).opts(title="Humidity (%) of Cities across the World")

# Display the map
city_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
city_df = city_data_df.dropna()

# Drop any rows with null values
cleaned_df = city_df.loc[(city_df["Max Temp"] > 21) & (city_df["Max Temp"] < 27) & (city_df["Wind Speed"]<4.5) & (city_df["Cloudiness"] <50), :]

# Display sample data
cleaned_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,isna,25.2934,32.5540,26.01,37,0,3.63,EG,1695351573
23,23,antonio enes,-16.2325,39.9086,21.93,91,1,1.54,MZ,1695351575
50,50,kita,13.0349,-9.4895,22.96,89,41,0.15,ML,1695351583
76,76,nova vicosa,-17.8919,-39.3719,22.33,80,3,3.61,BR,1695351589
82,82,vila velha,-20.3297,-40.2925,21.97,90,20,1.54,BR,1695351591


### Step 3: Create a new DataFrame called `hotel_df`.

In [29]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df.copy()
hotel_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,isna,25.2934,32.5540,26.01,37,0,3.63,EG,1695351573
23,23,antonio enes,-16.2325,39.9086,21.93,91,1,1.54,MZ,1695351575
50,50,kita,13.0349,-9.4895,22.96,89,41,0.15,ML,1695351583
76,76,nova vicosa,-17.8919,-39.3719,22.33,80,3,3.61,BR,1695351589
82,82,vila velha,-20.3297,-40.2925,21.97,90,20,1.54,BR,1695351591


In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.loc[:, 'Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,isna,EG,25.2934,32.5540,37,
23,antonio enes,MZ,-16.2325,39.9086,91,
50,kita,ML,13.0349,-9.4895,89,
76,nova vicosa,BR,-17.8919,-39.3719,80,
82,vila velha,BR,-20.3297,-40.2925,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":1,
    "apiKey":geoapify_key,  
    "format": "json",
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:({lng},{lat},{radius})"
    params["bias"] = f"proximity:({lng},{lat})"
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
isna - nearest hotel: No hotel found
antonio enes - nearest hotel: No hotel found
kita - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: No hotel found
tomatlan - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
kirkwood - nearest hotel: No hotel found
gebeit - nearest hotel: No hotel found
dailekh - nearest hotel: No hotel found
mongagua - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
itaituba - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
tarija - nearest hotel: No hotel found
isla mujeres - nearest hotel: No hotel found
erdemli - nearest hotel: No hotel found
chokwe - nearest hotel: No hotel found
libreville - nearest hotel: No hotel found
koysinceq - nearest hotel: No hotel found
cannonvale - nearest hotel: No hotel found
t

,City,Country,Lat,Lng,Humidity,Hotel Name
16,isna,EG,25.2934,32.5540,37,No hotel found
23,antonio enes,MZ,-16.2325,39.9086,91,No hotel found
50,kita,ML,13.0349,-9.4895,89,No hotel found
76,nova vicosa,BR,-17.8919,-39.3719,80,No hotel found
82,vila velha,BR,-20.3297,-40.2925,90,No hotel found
89,sao felix do xingu,BR,-6.6447,-51.9950,66,No hotel found
97,tomatlan,MX,19.9333,-105.2500,88,No hotel found
108,puerto ayora,EC,-0.7393,-90.3518,90,No hotel found
117,kirkwood,US,38.5834,-90.4068,87,No hotel found
132,gebeit,SD,21.0667,36.3167,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles= "OSM",
    frame_width = 600,
    frame_height = 600, 
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)